# UCS, DIJKSTRA Y DLS ALGORITHMS

In [ ]:
import heapq as hq
import math
import graphviz as gv

### Creacion y Lectura del archivo

In [ ]:
%%file g.in
1 7 2 2 3 4 4 6
0 7 2 4 3 4 5 3 6 2
0 2 1 4
0 4 1 4 5 9 7 3
0 6 6 9 7 9
1 3 3 9 6 8 7 4
1 2 4 9 5 8
3 3 4 9 5 4

In [ ]:
with open("g.in") as f:
    G = []
    for line in f:
        if line.startswith("-"):
            G.append([])
        else:
            a = line.split()
            G.append([(int(a[i]), float(a[i + 1])) for i in range(0, len(a), 2)])
print("[", end="")
for l in G:
    print(f"{l}")
print("]")

### Graficado de un Grafo

In [ ]:
def drawGraph(G, path=[], h=[], directed=False):
    g = gv.Digraph("nombre") if directed else gv.Graph("nombre")
    n = len(G)

    for u, hi in enumerate(h):
        g.node(str(u), f"{u}({hi})")

    done = set()
    for v, u in enumerate(path):
        if u != -1:
            g.edge(str(u), str(v), dir="forward", penwidth="2", color="orange")
            done.add((u, v))
            done.add((v, u))
    for u in range(n):
        g.node(str(u))
        for v, w in G[u]:
            if not (u, v) in done:
                done.add((u, v))
                done.add((v, u))
                g.edge(str(u), str(v), str(w))

    return g

In [ ]:
print(G)

In [ ]:
aux = []
drawGraph(G, h=aux)

### Implementacion del algoritmo de Dijkstra o UCS ( RECORRIDO POR COSTO UNIFORME )

In [1]:
def dijkstra(G, s):
    n = len(G)
    visited = [False] * n
    path = [-1] * n
    cost = [math.inf] * n
    cost[s] = 0

    pq = []
    hq.heappush(pq, (0, s))
    while pq:
        _, u = hq.heappop(pq)
        visited[u] = True
        for v, w in G[u]:
            f = cost[u] + w
            if not visited[v] and f < cost[v]:
                cost[v] = f
                path[v] = u
                hq.heappush(pq, (f, v))
    return path, cost

In [ ]:
path_disjktra, cost_disjktra = dijkstra(G, 7)
print(f"El path del grafo con Dijkstra es --->  {path_disjktra}")
print(f"El cost del grafo con Dijkstra es --->  {cost_disjktra}")

In [ ]:
drawGraph(G, path=path_disjktra, h=aux)

### DLS ( RECORRIDO EN PROFUNDIDAD LIMITADA )

In [ ]:
def dls(G, s, L): # depth limited search (EVADE PESOS)
  n = len(G)
  visited = [False]*n
  path = [-1]*n

  def _f(u,l):
    if l > 0:
      visited[u] = True
      for v,w in G[u]:
        if not visited[v]:
          path[v] = u
          _f(v, l - 1)
  _f(s, L)
  return path

In [ ]:
path_dls = dls(G, 1, 4)
print(path_dls)

In [ ]:
drawGraph(G, path=path_dls, h=aux)

### IDS ( RECORRIDO EN PROFUNDIDAD ITERATIVA )

### Creacion de un grafo para IDS

In [ ]:
%%file 0503.al
1 4
-
-
2 6 17
7
2
10
0 8
4
8
-
8 14
16
9 17
15
11 17
15 17
13

### Lectura del grafo y representacion en listas

In [ ]:
with open("0503.al") as f:
  G_3 = []
  for line in f:
    if line == "-\n":
      G_3.append([])
    else:
      G_3.append([int(x) for x in line.split()])

for x in G_3:
  print(x)

### OTRA VERSION DEL GRAFICADO DE GRAFOS

In [ ]:
def drawG_al(G, directed=False, weighted=False, path=[], layout="sfdp"):
  graph = gv.Digraph("felicidad") if directed else gv.Graph("alegria")
  graph.graph_attr["layout"] = layout
  graph.edge_attr["color"] = "gray"
  graph.node_attr["color"] = "orangered"
  graph.node_attr["width"] = "0.1"
  graph.node_attr["height"] = "0.1"
  graph.node_attr["fontsize"] = "8"
  graph.node_attr["fontcolor"] = "mediumslateblue"
  graph.node_attr["fontname"] = "monospace"
  graph.edge_attr["fontsize"] = "8"
  graph.edge_attr["fontname"] = "monospace"
  n = len(G)
  added = set()
  for v, u in enumerate(path):
    if u != -1:
      if weighted:
        for vi, w in G[u]:
          if vi == v:
            break
        graph.edge(str(u), str(v), str(w), dir="forward", penwidth="2", color="orange")
      else:
        graph.edge(str(u), str(v), dir="forward", penwidth="2", color="orange")
      added.add(f"{u},{v}")
      added.add(f"{v},{u}")
  for u in range(n):
    for edge in G[u]:
      if weighted:
        v, w = edge
      else:
        v = edge
      draw = False
      if not directed and not f"{u},{v}" in added:
        added.add(f"{u},{v}")
        added.add(f"{v},{u}")
        draw = True
      elif directed and not f"{u},{v}" in added:
        added.add(f"{u},{v}")
        draw = True
      if draw:
        if weighted:
          graph.edge(str(u), str(v), str(w))
        else:
          graph.edge(str(u), str(v))
  return graph

### COMO IDS USA A DLS ---> CREAMOS UNA VERSION DE DLS SIN USAR PESOS

In [ ]:
def dls2(G, s, L):
  n = len(G)
  visited = [False]*n
  path = [-1]*n

  def _dls2(u, L):
    if L > 0 and not visited[u]:
      visited[u] = True
      for v in G[u]:
        if not visited[v]:
          path[v] = u
          _dls2(v, L - 1)

  _dls2(s, L)
  return path

In [ ]:
def ids(G, start, target):
  n = len(G)
  for limit in range(n):
    path = dls2(G, start, limit)
    if path[target] != -1:
      break
  return path

In [ ]:
drawG_al(G_3, directed=True, layout="neato")

In [ ]:
path_ids_1 = ids(G_3, start = 12, target = 7)
print(path_ids_1)
drawG_al(G_3, directed = True, path = path_ids_1, layout = "neato")

In [ ]:
path_ids_2 = ids(G_3, start = 12, target = 10)
print(path_ids_2)
drawG_al(G_3, directed = True, path = path_ids_2, layout = "neato")

### A Star (A')

In [ ]:
def a_star(G, s, t, h):
    n = len(G)
    g = [math.inf]*n
    visited = [False]*n
    f = [math.inf]*n
    path = [-1]*n

    g[s] = 0
    f[s] = h[s]
    q = []
    hq.heappush(q, (f[s], s))
    while q:
        _, n = hq.heappop(q)
        if not visited[n]:
            visited[n] = True
            if n == t:
                break
            for v, w in G[n]:
                if not visited[v] and g[n] + w < g[v]:
                    path[v] = n
                    g[v] = g[n] + w
                    f[v] = g[v] + h[v]
                    hq.heappush(q, (f[v], v))

    return path

### Preparando el entorno para el caso de A star

In [ ]:
def manhattan(a, b):
    x0, y0 = a
    x1, y1 = b
    return abs(x0 - x1) + abs(y0 - y1)

In [ ]:
tamano = 6 # Tamaño de la la lista de listas tamano x tamano
G_2 = [[] for i in range(tamano * tamano)] # Crea una lista de listas de (tamano x tamano) elementos
end = (5, 5) # Variable par para el calculo de h
h = [] # Peso especiales por vertice, calculados por manhattan

### La siguiente funcion genera los datos del grafo

In [ ]:
for i in range(tamano):
    for j in range(tamano):
        idx = i * tamano + j
        nbrs = [(i-1, j-1, 2), (i-1, j, 1), (i-1, j+1, 2),
                (  i, j-1, 1),              (  i, j+1, 1),
                (i+1, j-1, 2), (i+1, j, 1), (i+1, j+1, 2)]
        for i0, j0, w in nbrs:
            if i0 < 0 or i0 >= tamano or j0 < 0 or j0 >= tamano:
                continue
            idx0 = i0 * tamano + j0
            G_2[idx].append((idx0, w))
        h.append(manhattan((i, j), end))

In [ ]:
print(G_2)

In [ ]:
print(h) # Imprime los pesos especiales de cada nodo

In [ ]:
drawGraph(G_2, h=h)

In [ ]:
path_Astar = a_star(G_2, 7, 35, h)
print(path_Astar)

In [ ]:
drawGraph(G_2, path = path_Astar, h = h)